In [1]:
!python -m pip install --upgrade pip

!pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
get_ipython().system_raw("mlflow ui --port 5090 &")

In [4]:
from pyngrok import ngrok

In [5]:
ngrok.kill()

In [6]:
!ngrok config add-authtoken 2sViGViQ9YXhZ93JPXnt9zp2kSn_5tS656cBtfCP4acM4JTkb

Authtoken saved to configuration file: /home/workbench/.config/ngrok/ngrok.yml


In [7]:
ngrok_tunnel = ngrok.connect(addr="5090", proto="http", bind_tls=True)

In [8]:
print("MLflow UI ", ngrok_tunnel.public_url)

MLflow UI  https://4eba-67-254-220-141.ngrok-free.app


In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.pytorch


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/workbench/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/workbench/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/workbench/.local/lib/python3.10

In [10]:
mlflow.set_experiment(experiment_name="mlflow pytorch demo")
    
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')
dataset.describe()
dataset.head()

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

Xtrain_

ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

ytrain_

Xtrain_.shape, ytrain_.shape

Xtest_.shape, ytest_.shape

input_size=2
output_size=2
hidden_size=10

class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

model = Net()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

epochs = 150

for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

mlflow.end_run()
with mlflow.start_run() as run:
    mlflow.log_param("epochs", 150)
    mlflow.pytorch.log_model(model, "models")
    mlflow.log_metric("loss", loss.item())

RuntimeError: Numpy is not available

In [12]:
ngrok.kill()